In [2]:
import torch

In [3]:
x = torch.empty(5, 3)
print(x)

tensor([[-2.0544e+21,  4.5856e-41, -2.0544e+21],
        [ 4.5856e-41, -4.2454e+20,  4.5856e-41],
        [-4.2456e+20,  4.5856e-41, -6.7113e+14],
        [ 4.5856e-41, -4.2494e+20,  4.5856e-41],
        [-4.2618e+20,  4.5856e-41, -6.3124e+14]])


In [4]:
x = torch.rand(5, 3)
print(x)

tensor([[0.0203, 0.0820, 0.6141],
        [0.7359, 0.6257, 0.2334],
        [0.3597, 0.6205, 0.4836],
        [0.9665, 0.6889, 0.2108],
        [0.6212, 0.3662, 0.4566]])


In [5]:
torch.cuda.is_available()

True

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([[1.0203, 1.0820, 1.6141],
        [1.7359, 1.6257, 1.2334],
        [1.3597, 1.6205, 1.4836],
        [1.9665, 1.6889, 1.2108],
        [1.6212, 1.3662, 1.4566]], device='cuda:0')
tensor([[1.0203, 1.0820, 1.6141],
        [1.7359, 1.6257, 1.2334],
        [1.3597, 1.6205, 1.4836],
        [1.9665, 1.6889, 1.2108],
        [1.6212, 1.3662, 1.4566]], dtype=torch.float64)


In [7]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [8]:
y = x + 2

In [12]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward>) tensor(27., grad_fn=<MeanBackward1>)


In [13]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [14]:
out.backward()

In [15]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [22]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([  454.1067, -1101.2789,  -305.1438], grad_fn=<MulBackward>)


In [23]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([ 102.4000, 1024.0000,    0.1024])


In [24]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [26]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [27]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0589,  0.0473,  0.0806, -0.0861,  0.0059, -0.0129, -0.0039,  0.0797,
         -0.0126,  0.0946]], grad_fn=<ThAddmmBackward>)


In [28]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [29]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8365, grad_fn=<MseLossBackward>)


In [30]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [31]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0101,  0.0042,  0.0036, -0.0192,  0.0303,  0.0011])


In [32]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update